In [1]:
import warnings
warnings.resetwarnings()

import scprep
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import random

import os

from tqdm import tqdm

from deepimpute.multinet import MultiNet

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

2023-08-07 18:01:03.830741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 18:01:14.677760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_data_for_i(i):
    original_ = pd.read_csv('../../data/cell_simulation_10000/data.csv.gz', index_col=0)
    df_ = pd.read_csv('../../data/cell_simulation_10000/drp_{}0.csv.gz'.format(i), index_col=0)
    df_.index = [int(i) for i in df_.index]
    df_.columns = [int(i) for i in df_.columns]

    original_.columns = df_.columns
    original_.index = df_.index

    n = original_.size
    original_val = original_.values.copy()
    t = list(np.ndindex(original_.shape))
    random.Random(42).shuffle(t)

    mask = t[:int(len(t)/10 * i)]

    thr = np.sum(np.sign(df_)) > 0
    original_ = original_.loc[:, list(thr)]
    df_ = df_.loc[:, list(thr)]

    # original = original_.values
    original = np.log(original_+1)

    # df = df_.values
    df = np.log(df_+1)

    tmp = pd.DataFrame(thr)
    remove = [int(i) for i in tmp[tmp[0] == False].index]
    mask = [i for i in mask if i[1] not in remove]
    
    return df_, mask, original

In [3]:
mses = {}
corrs = {}
method = 'DeepImpute'

for i in (range(9)):
    print(i)
    df_, mask, original = get_data_for_i(i+1)

    multinet = MultiNet() 
    multinet.fit(df_)
    
    pred = multinet.predict(df_)
    pred = np.log(pred.values+1)
    
    pred = pd.DataFrame(pred, index=df_.index, columns=df_.columns)
    pred.to_csv('/export/scratch/inoue019/cell_result/{}_{}.csv.gz'.format(method, i), compression='gzip')

    origin = np.array([original.loc[i] for i in mask])
    predict = np.array([pred.loc[i] for i in mask])

    mses[i] = mse(origin, predict)
    corrs[i] = np.corrcoef(origin, predict)[0][1]

0
Using all the cores (20)
Input dataset is 10000 cells (rows) and 5000 genes (columns)
First 3 rows and columns:
   0  1  2
0  0  2  0
1  1  5  1
2  1  0  1
5120 genes selected for imputation
Net 0: 566 predictors, 512 targets
Net 1: 582 predictors, 512 targets
Net 2: 525 predictors, 512 targets
Net 3: 556 predictors, 512 targets
Net 4: 537 predictors, 512 targets
Net 5: 574 predictors, 512 targets
Net 6: 590 predictors, 512 targets
Net 7: 566 predictors, 512 targets
Net 8: 567 predictors, 512 targets
Net 9: 585 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]


2023-08-07 18:02:41.799214: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/inoue019/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 10000 cells
Epoch 1/500
149/149 [==============================] - 5s 11ms/step - loss: 3.1553 - dense_10_loss: 0.3075 - dense_11_loss: 0.3191 - dense_12_loss: 0.3233 - dense_13_loss: 0.3191 - dense_14_loss: 0.2760 - dense_15_loss: 0.3229 - dense_16_loss: 0.3168 - dense_17_loss: 0.3229 - dense_18_loss: 0.2865 - dense_19_loss: 0.3613 - val_loss: 1.6341 - val_dense_10_loss: 0.1600 - val_dense_11_loss: 0.1644 - val_dense_12_loss: 0.1579 - val_dense_13_loss: 0.1711 - val_dense_14_loss: 0.1483 - val_dense_15_loss: 0.1684 - val_dense_16_loss: 0.1665 - val_dense_17_loss: 0.1631 - val_dense_18_loss: 0.1584 - val_dense_19_loss: 0.1760
Epoch 2/500
149/149 [==============================] - 1s 8ms/step - loss: 1.8642 - dense_10_loss: 0.1808 - dense_11_loss: 0.1898 - dense_12_loss: 0.1847 - dense_13_loss: 0.1926 - dense_14_loss: 0.1701 - dense_15_loss: 0.1915 - dense_16_loss: 0.1883 - dense_17_loss: 0.1864 - dense_18_loss: 0.1785 - dense_19_loss: 0.2015 - val_loss: 1.4323 - val_dense_

149/149 [==============================] - 1s 8ms/step - loss: 1.1789 - dense_10_loss: 0.1140 - dense_11_loss: 0.1218 - dense_12_loss: 0.1173 - dense_13_loss: 0.1220 - dense_14_loss: 0.1109 - dense_15_loss: 0.1183 - dense_16_loss: 0.1183 - dense_17_loss: 0.1167 - dense_18_loss: 0.1150 - dense_19_loss: 0.1247 - val_loss: 1.0947 - val_dense_10_loss: 0.1073 - val_dense_11_loss: 0.1117 - val_dense_12_loss: 0.1080 - val_dense_13_loss: 0.1134 - val_dense_14_loss: 0.1033 - val_dense_15_loss: 0.1100 - val_dense_16_loss: 0.1099 - val_dense_17_loss: 0.1085 - val_dense_18_loss: 0.1074 - val_dense_19_loss: 0.1153
Epoch 15/500
149/149 [==============================] - 1s 8ms/step - loss: 1.1650 - dense_10_loss: 0.1120 - dense_11_loss: 0.1203 - dense_12_loss: 0.1159 - dense_13_loss: 0.1207 - dense_14_loss: 0.1099 - dense_15_loss: 0.1173 - dense_16_loss: 0.1173 - dense_17_loss: 0.1154 - dense_18_loss: 0.1136 - dense_19_loss: 0.1226 - val_loss: 1.0937 - val_dense_10_loss: 0.1068 - val_dense_11_loss: 

149/149 [==============================] - 1s 8ms/step - loss: 1.0684 - dense_10_loss: 0.1026 - dense_11_loss: 0.1102 - dense_12_loss: 0.1062 - dense_13_loss: 0.1106 - dense_14_loss: 0.1014 - dense_15_loss: 0.1071 - dense_16_loss: 0.1073 - dense_17_loss: 0.1060 - dense_18_loss: 0.1048 - dense_19_loss: 0.1123 - val_loss: 1.0530 - val_dense_10_loss: 0.1027 - val_dense_11_loss: 0.1076 - val_dense_12_loss: 0.1033 - val_dense_13_loss: 0.1087 - val_dense_14_loss: 0.0997 - val_dense_15_loss: 0.1051 - val_dense_16_loss: 0.1054 - val_dense_17_loss: 0.1064 - val_dense_18_loss: 0.1035 - val_dense_19_loss: 0.1105
Epoch 28/500
149/149 [==============================] - 1s 7ms/step - loss: 1.0641 - dense_10_loss: 0.1021 - dense_11_loss: 0.1094 - dense_12_loss: 0.1064 - dense_13_loss: 0.1098 - dense_14_loss: 0.1007 - dense_15_loss: 0.1066 - dense_16_loss: 0.1068 - dense_17_loss: 0.1058 - dense_18_loss: 0.1045 - dense_19_loss: 0.1120 - val_loss: 1.0493 - val_dense_10_loss: 0.1029 - val_dense_11_loss: 

...

Fitting with 10000 cells
Epoch 1/500
149/149 [==============================] - 2s 8ms/step - loss: 0.8847 - dense_127_loss: 0.1804 - dense_128_loss: 0.1667 - dense_129_loss: 0.1664 - dense_130_loss: 0.1884 - dense_131_loss: 0.1828 - val_loss: 0.3873 - val_dense_127_loss: 0.0832 - val_dense_128_loss: 0.0753 - val_dense_129_loss: 0.0732 - val_dense_130_loss: 0.0791 - val_dense_131_loss: 0.0766
Epoch 2/500
149/149 [==============================] - 1s 7ms/step - loss: 0.4295 - dense_127_loss: 0.0895 - dense_128_loss: 0.0837 - dense_129_loss: 0.0814 - dense_130_loss: 0.0871 - dense_131_loss: 0.0878 - val_loss: 0.3536 - val_dense_127_loss: 0.0755 - val_dense_128_loss: 0.0687 - val_dense_129_loss: 0.0666 - val_dense_130_loss: 0.0722 - val_dense_131_loss: 0.0707
Epoch 3/500
149/149 [==============================] - 1s 7ms/step - loss: 0.3890 - dense_127_loss: 0.0808 - dense_128_loss: 0.0761 - dense_129_loss: 0.0736 - dense_130_loss: 0.0787 - dense_131_loss: 0.0799 - val_loss: 0.3355 - val_d

Epoch 23/500
149/149 [==============================] - 1s 7ms/step - loss: 0.1926 - dense_127_loss: 0.0393 - dense_128_loss: 0.0369 - dense_129_loss: 0.0364 - dense_130_loss: 0.0392 - dense_131_loss: 0.0408 - val_loss: 0.2545 - val_dense_127_loss: 0.0526 - val_dense_128_loss: 0.0503 - val_dense_129_loss: 0.0480 - val_dense_130_loss: 0.0514 - val_dense_131_loss: 0.0523
Epoch 24/500
149/149 [==============================] - 1s 7ms/step - loss: 0.1891 - dense_127_loss: 0.0386 - dense_128_loss: 0.0363 - dense_129_loss: 0.0357 - dense_130_loss: 0.0385 - dense_131_loss: 0.0400 - val_loss: 0.2542 - val_dense_127_loss: 0.0523 - val_dense_128_loss: 0.0503 - val_dense_129_loss: 0.0481 - val_dense_130_loss: 0.0511 - val_dense_131_loss: 0.0523
Epoch 25/500
149/149 [==============================] - 1s 7ms/step - loss: 0.1859 - dense_127_loss: 0.0380 - dense_128_loss: 0.0356 - dense_129_loss: 0.0351 - dense_130_loss: 0.0378 - dense_131_loss: 0.0394 - val_loss: 0.2538 - val_dense_127_loss: 0.0522 

/home/inoue019/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 10000 cells
Epoch 1/500
149/149 [==============================] - 5s 8ms/step - loss: 0.5913 - dense_136_loss: 0.1424 - dense_137_loss: 0.1390 - dense_138_loss: 0.1351 - dense_139_loss: 0.1748 - val_loss: 0.2766 - val_dense_136_loss: 0.0710 - val_dense_137_loss: 0.0648 - val_dense_138_loss: 0.0608 - val_dense_139_loss: 0.0801
Epoch 2/500
149/149 [==============================] - 1s 6ms/step - loss: 0.2056 - dense_136_loss: 0.0523 - dense_137_loss: 0.0464 - dense_138_loss: 0.0495 - dense_139_loss: 0.0574 - val_loss: 0.1639 - val_dense_136_loss: 0.0445 - val_dense_137_loss: 0.0395 - val_dense_138_loss: 0.0386 - val_dense_139_loss: 0.0413
Epoch 3/500
149/149 [==============================] - 1s 7ms/step - loss: 0.1776 - dense_136_loss: 0.0456 - dense_137_loss: 0.0410 - dense_138_loss: 0.0432 - dense_139_loss: 0.0478 - val_loss: 0.1595 - val_dense_136_loss: 0.0427 - val_dense_137_loss: 0.0387 - val_dense_138_loss: 0.0377 - val_dense_139_loss: 0.0404
Epoch 4/500
149/149 [===

Epoch 27/500
149/149 [==============================] - 1s 6ms/step - loss: 0.0774 - dense_136_loss: 0.0198 - dense_137_loss: 0.0183 - dense_138_loss: 0.0192 - dense_139_loss: 0.0202 - val_loss: 0.1275 - val_dense_136_loss: 0.0342 - val_dense_137_loss: 0.0305 - val_dense_138_loss: 0.0310 - val_dense_139_loss: 0.0319
Epoch 28/500
149/149 [==============================] - 1s 7ms/step - loss: 0.0759 - dense_136_loss: 0.0195 - dense_137_loss: 0.0179 - dense_138_loss: 0.0187 - dense_139_loss: 0.0197 - val_loss: 0.1273 - val_dense_136_loss: 0.0341 - val_dense_137_loss: 0.0304 - val_dense_138_loss: 0.0309 - val_dense_139_loss: 0.0318
Epoch 29/500
149/149 [==============================] - 1s 7ms/step - loss: 0.0743 - dense_136_loss: 0.0190 - dense_137_loss: 0.0175 - dense_138_loss: 0.0184 - dense_139_loss: 0.0194 - val_loss: 0.1272 - val_dense_136_loss: 0.0341 - val_dense_137_loss: 0.0305 - val_dense_138_loss: 0.0309 - val_dense_139_loss: 0.0317
Epoch 30/500
149/149 [========================

In [4]:
pd.DataFrame(mses.values()).T

,0,1,2,3,4,5,6,7,8
0,0.965223,0.982766,0.916176,0.841931,0.763632,0.762417,0.63168,0.578651,0.538103


In [5]:
pd.DataFrame(corrs.values()).T

,0,1,2,3,4,5,6,7,8
0,0.664645,0.660516,0.567511,0.528247,0.516393,0.513799,0.53061,0.542326,0.545422
